# Eat Safe, Love

## Notebook Set Up

In [18]:
!pip install pymongo

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
client = MongoClient(port=27017)

In [3]:
print(client.list_database_names())

['admin', 'config', 'local', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = client['uk_food']

In [5]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [6]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [7]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
print(f"There are {establishments.count_documents(query)} documents with a 'Hygiene' score of 20.")

# Display the first document in the results using pprint
results = establishments.find(query)
print("")
print("The first document is:")
print("-"*50)
pprint(results[0])


There are 41 documents with a 'Hygiene' score of 20.

The first document is:
--------------------------------------------------
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': '0',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('667a2ce300b54d54aa82334e'),
 'geocode': {'latitude': 50.769705, 'longitude': 0.27694},
 'links': [{'href': 'https:/

In [8]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.json_normalize(results)

# Export DataFrame to csv file
hygiene_df.to_csv("csv_outputs/hygiene_df.csv", index=False, header=True)

# Display the number of rows in the DataFrame
print(f"The number of rows in this DataFrame is: {len(hygiene_df)}")
print("")

# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

OSError: Cannot save file into a non-existent directory: 'csv_outputs'

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName":{'$regex': "London"}, "RatingValue" : {'$gte':4}}

# Use count_documents to display the number of documents in the result
print(f"There are {establishments.count_documents(query)} documents for establishments within 'London' with a 'RatingValue' >= 4.")

# Display the first document in the results using pprint
results = establishments.find(query)
print("")
print("The first document is:")
print("-"*50)
pprint(results[0])

In [9]:
# Convert the result to a Pandas DataFrame
london_rating = pd.json_normalize(results)

# Export DataFrame to csv file
london_rating.to_csv("csv_outputs/london_rating.csv", index=False, header=True)

# Display the number of rows in the DataFrame
print(f"The number of rows in this DataFrame is: {len(london_rating)}")
print("")

# Display the first 10 rows of the DataFrame
london_rating.head(10)

OSError: Cannot save file into a non-existent directory: 'csv_outputs'

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [10]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

# Extract the 'geocode' field from the document with 'BusinessName' 'Penang Flavours'
lat_long = establishments.find_one({"BusinessName":"Penang Flavours"},{"geocode":1})


degree_search = 0.01
# Assign the lat and long values to variables
latitude = lat_long['geocode']['latitude']
longitude = lat_long['geocode']['longitude']

# Calculate the lat/long search area by +/- the 'degree_search' from the lat/long variables
query = {"geocode.latitude":{'$gte': latitude-degree_search, '$lte': latitude+degree_search},
        "geocode.longitude":{'$gte': longitude-degree_search, '$lte': longitude+degree_search},
        "RatingValue":5}
sort = [("scores.Hygiene",1)]
limit = 5

# Print the results
results = list(establishments.find(query).sort(sort).limit(limit))
for r in results:
    pprint(r)

In [11]:
# Convert result to Pandas DataFrame
near_penang_df = pd.json_normalize(results)

# Export DataFrame to csv file
near_penang_df.to_csv("csv_outputs/near_penang_df.csv", index=False, header=True)

near_penang_df

OSError: Cannot save file into a non-existent directory: 'csv_outputs'

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [12]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Pipeline
match = {'$match': {"scores.Hygiene":0}}

group = {'$group': {"_id" : "$LocalAuthorityName",
                   "count": {'$sum':1}}}

sort = {'$sort':{"count":-1}}

pipeline = [match, group, sort]

results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f"There are {len(results)} documents in the result.")

# Print the first 10 results
print("")
print("The first 10 documents are:")
print("-"*50)
pprint(results[0:10])

There are 55 documents in the result.

The first 10 documents are:
--------------------------------------------------
[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [13]:
# Convert the result to a Pandas DataFrame
agg_df = pd.json_normalize(results)

# Export DataFrame to csv file
agg_df.to_csv("csv_outputs/aggregate_df.csv", index=False, header=True)

# Display the number of rows in the DataFrame
print(f"The number of rows in this DataFrame is: {len(agg_df)}")
print("")

# Display the first 10 rows of the DataFrame
agg_df.head(10)

OSError: Cannot save file into a non-existent directory: 'csv_outputs'